# Load data to InfluxDB

InfluxDB is a non relational data base specific for time series data sets. It has meany features, one is that can handle SQL like queries.

Here are shown the key steps to load the data una array of jsons to a measurement or **table** in InfluxDB

In [1]:
from datetime import datetime
import json

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

Now we perform the actual connection to the db

In [2]:
url = "http://influxdb:8086"
token = "my-admin-token"
org = "my-org"
bucket = "my-bucket"

with InfluxDBClient(url=url, token=token, org=org) as client:
    if not client.ping():
        raise AttributeError("Can't setup connection to InfluxDB")
    else:
        print(f'Working connection on {client.url}')

Working connection on http://influxdb:8086


Now we load the json file and setup the payload structure to be load into this Influxdb.

In [3]:
starklink_datafile = 'starlink_historical_data.json'

with open(starklink_datafile, 'r') as data_file:
    json_data = data_file.read()

starklink_data = json.loads(json_data)

points = [
    Point('sat_pos_' + data['id'])\
    .field('lat', float(data['latitude']))\
    .field('lon', float(data['longitude']))\
    .time(data['spaceTrack']['CREATION_DATE'])
    
    for data in starklink_data if data['latitude'] and data['longitude'] ]

As we can see here, the payload consist on a list of dictionaries, where the key fields on these dictionaries are

    1. measurement: this is like the SQL DB where all the data will be allocated inside the DB.
    2. time: this is a ISO8601 or similar datetime string
    3. fields: points to a dict of the actual pair key-value of data stored
  
As an example


In [4]:
display(points[100].__dict__)

{'_tags': {},
 '_fields': {'lat': -28.351511934981414, 'lon': 41.0},
 '_name': 'sat_pos_5eed7714096e5900069856a2',
 '_time': '2021-01-26T02:30:00',
 '_write_precision': 'ns'}

Now with the payload constructed, we can do the actual write to the DB and then query for these measurements

In [5]:
with InfluxDBClient(url=url, token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    write_api.write(bucket=bucket, record=points)


In [6]:
with InfluxDBClient(url=url, token=token, org=org) as client:
    query_api = client.query_api()
    csv_result = query_api.query_csv(
        """
        from(bucket: "my-bucket")
          |> range(start: -3y)
          |> filter(fn: (r) => r["_measurement"] == "sat_pos_5eed7714096e5900069856a2")
          |> filter(fn: (r) => r["_field"] == "lon")
        """)
    
for row in csv_result:
    print(', '.join(row))

#datatype, string, long, dateTime:RFC3339, dateTime:RFC3339, dateTime:RFC3339, double, string, string
#group, false, false, true, true, false, false, true, true
#default, _result, , , , , , , 
, result, table, _start, _stop, _time, _value, _field, _measurement
, , 0, 2019-01-20T09:35:18.8427593Z, 2022-01-20T03:35:18.8427593Z, 2021-01-21T06:26:10Z, 155, lon, sat_pos_5eed7714096e5900069856a2
, , 0, 2019-01-20T09:35:18.8427593Z, 2022-01-20T03:35:18.8427593Z, 2021-01-26T02:30:00Z, 41, lon, sat_pos_5eed7714096e5900069856a2
, , 0, 2019-01-20T09:35:18.8427593Z, 2022-01-20T03:35:18.8427593Z, 2021-01-26T06:26:10Z, 5, lon, sat_pos_5eed7714096e5900069856a2

